In [1]:
import pickle
import re
from tqdm import tqdm

In the block below we are reading the raw data from articles and tables.

In [2]:
rawArticles = pickle.load(open("NewsDataset/final_article_tables.pkl","rb"))
rawTables = pickle.load(open("NewsDataset/final_dict_tables.pkl","rb"))

In the block next, we are formatting the articles data by removing links, multiples spaces and special characters. Each article has several links. Then we get each one. Finally, we read all text of each link.

In [9]:
corpusArticles = []

for i in tqdm(rawArticles.keys()):
    
    keyArticle = i
     
    dataArticle = rawArticles[keyArticle]

    totalArticles = len(dataArticle)

    for i in range(totalArticles):
    
        fullText = str(dataArticle[i][1])
    
        #removing special characters
        formattedText = re.sub('[^A-Za-z]+',' ',fullText)
    
        #removing left and right spaces
        formattedText = formattedText.lstrip()
        formattedText = formattedText.rstrip()
    
        corpusArticles.append([keyArticle,formattedText])

100%|██████████| 90000/90000 [12:27<00:00, 120.33it/s]


In the next block, we remove empty values from articles.

In [37]:
for article in corpusArticles:
    if (article[1] == ''):
        corpusArticles.remove(article)

In the block next, we are reading and formatting the raw table data. Each indice may have multiple tables. So, we collect each table separately. Next, we add tables in the corpus.

In [3]:
corpusTable = []

for i in tqdm(rawTables.keys()):
    
    dataTable = rawTables[i]
    
    totalTalbes = len(dataTable)
    
    for j in range(totalTalbes):
        
        keyTable = dataTable[j][0][0]
        
        keyCaption = dataTable[j][0][1]
        
        tableText = str(dataTable[j][1:])
    
        fullTextTable = re.sub('[^A-Za-z]+',' ',tableText)
        
        #removing left and right spaces
        fullTextTable = fullTextTable.lstrip()
        fullTextTable = fullTextTable.rstrip()
        
        corpusTable.append([keyTable,keyCaption,fullTextTable])

100%|██████████| 90000/90000 [02:39<00:00, 563.13it/s] 


tf-idf method

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
corpusTable[:2,1:,:2]

TypeError: list indices must be integers or slices, not tuple